**ASSISTA A AULA "Heroku e Metabase: entendendo suas funcionalidades " PARA ACESSAR O METABASE**



É essencial no dia a dia de um Analista, Cientista e Engenheiro de Dados analisarmos os dados brutos através do SQL e responder perguntas importantes do nosso dia a dia e sobre o negócio. Para responder e observar esses problemas através do dados é indispensável a utilização do SQL e uma ferramenta de BI (Metabase) para disponibilizar os dados de forma rápida e prática para os tomadores de decisão.

Aqui temos a Lojinha de Presentes da DNC, que é responsável por vender apetrechos, aparelhos, presentinhos e outros itens da mesma categoria para alunos da DNC. Você como um consultor, deve analisar os dados brutos de venda e responder algumas questões chaves para que os tomadores de decisão tenham informação suficiente para decidir a continuidade de algum produto, se fazem ou não promoções e no final, fornecer um dashboard para que os mesmos consigam acompanhar os resultados dessas tomadas de decisão.

Primeiro analise os dados disponíveis e entenda como as diversas tabelas se comunicam entre sí. Usando SQL/ Metabase, responda as 5 questões abaixo: 

**1 - Quais os 5 produtos mais caros da tabela products?**

--Quais os 5 produtos mais caros da tabela products?


select title, price from products group by PRICE, title order by PRICE desc limit(5)

**2 - Quais os 5 produtos mais vendidos da loja?**

select P.TITLE, count(PRODUCT_ID) QTD from ORDERS
LEFT JOIN PRODUCTS P ON ORDERS.PRODUCT_ID = P.ID group by PRODUCT_ID order by QTD DESC LIMIT(5)

**3 - Quais os 5 produtos da loja mais vendidos em 2020?**

select 
     PRODUCT_ID,COUNT(QUANTITY) as qtd
        
from ORDERS 
where year(cast(CREATED_AT as date)) = 2020
GROUP BY PRODUCT_ID, year(cast(CREATED_AT as date))
order by qtd desc limit(5)

**4 - Quais os 5 produtos mais avaliados?**

SELECT R.PRODUCT_ID, P.TITLE, COUNT(DISTINCT REVIEWER) AS contagem FROM REVIEWS AS R LEFT JOIN PRODUCTS AS P ON P.ID = R.PRODUCT_ID GROUP BY R.PRODUCT_ID, P.TITLE ORDER BY contagem DESC, PRICE asc LIMIT 5

**5 - De todas as avaliações, qual é a pessoa que mais avaliou algum produto com nota 5? (Em caso de empate, selecione por ordem alfabética: A- B- C-..)**

WITH PESSOAS AS (

    SELECT ID, NAME FROM PEOPLE


),

VENDAS AS (

    SELECT ID, PRODUCT_ID, USER_ID FROM ORDERS

),

PVENDAS AS (

   select COUNT(r.RATING) as Avaliacao, P.NAME as NOME,V.PRODUCT_ID, V.USER_ID  from PESSOAS P LEFT JOIN VENDAS V
   ON P.ID = USER_ID left join REVIEWS r on  V.PRODUCT_ID = r.PRODUCT_ID GROUP BY P.ID,V.PRODUCT_ID ORDER BY Avaliacao desc limit(10) 

)

select * from PVENDAS order by NOME 



**6 - Calcule o ticket médio por pessoa de todos os pedidos registrados na tabela ORDERS. (Ticket Médio = Total de vendas (Em U$S)/ Número de pedidos)?**

select user_id, sum(Total)/count(id),count(id) from ORDERS group by user_id

**7 - Construa um gráfico de linhas que apresente o número de pedidos e total de vendas em U$S no mesmo gráfico (Gráfico com 2 eixos y). Dica utilize esse link: https://www.metabase.com/learn/sql-questions/dates-in-sql#group-results-by-a-time-period**

WITH temp AS ( SELECT *, DATE_TRUNC('week', created_at) AS WEEK_DATE FROM ORDERS AS O)

SELECT WEEK_DATE AS period, COUNT(ID) AS NUM_ORDERS, SUM(TOTAL) AS TOTAL_CASH FROM temp GROUP BY period

**8 - O ticket médio por Mês de vendas? (Utilizar a dica da questão anterior)**

WITH temp AS ( SELECT *, DATE_TRUNC('month', created_at) AS MES_DATE FROM ORDERS AS O)

SELECT MES_DATE AS period, COUNT(ID) AS NUM_ORDERS, SUM(TOTAL) AS TOTAL_CASH FROM temp GROUP BY period

**9 - Top 5 mais bem avaliados produtos ( Em caso de empate, escolha o mais barato)**

SELECT A.PRODUCT_ID, B.TITLE, B.EAN, AVG(CAST(A.RATING AS DECIMAL)) AS RATING_AVG, B.PRICE FROM REVIEWS AS A LEFT JOIN PRODUCTS AS B ON A.PRODUCT_ID = B.ID GROUP BY A.PRODUCT_ID ORDER BY RATING_AVG DESC, B.PRICE ASC LIMIT 5

**10 - Agrupando por fornecedor, quais são os Top 10 fornecedores que mais geraram rendimento em Dezembro de 2019 (Considere rendimento como a soma total do valor do pedido e não a quantidade de pedidos daquele fornecedor).**

SELECT P.VENDOR, SUM(O.TOTAL) FROM ORDERS AS O LEFT JOIN PRODUCTS AS P ON P.ID = O.PRODUCT_ID 
WHERE YEAR(O.CREATED_AT) = 2019 AND MONTH(O.CREATED_AT) = 12 GROUP BY P.VENDOR ORDER BY SUM(O.TOTAL) DESC LIMIT 10

**11 - Mostre o total de pedidos distintos por categoria dos pedidos realizados no ano de 2019 ou posterior e que algum cliente tenha avaliado o produto com nota 4 ou maior.**

select category, count(distinct o.ID) Total_Pedidos  from orders o 
 left join PRODUCTS p on o.PRODUCT_ID=p.ID left join REVIEWS r on o.PRODUCT_ID=r.PRODUCT_ID
 where year(cast(o.CREATED_AT as date)) >= 2019 and r.RATING >= 4
 group by p.CATEGORY
 